Сделал Костин А.С. гр.8Е21. Все лежит там https://github.com/arskstn/LB2, csv-шки тоже

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBRegressor

In [ ]:
train = pd.read_csv('home-data-for-ml-course/train.csv')
test = pd.read_csv('home-data-for-ml-course/test.csv')

In [ ]:
missing = train.isnull().sum()
print("Пропущенные значения:\n", missing[missing > 0].sort_values(ascending=False))

In [ ]:
print("\nТипы данных:\n", train.dtypes)

In [ ]:
numeric_cols = train.select_dtypes(include=['int64', 'float64']).columns
train[numeric_cols] = train[numeric_cols].fillna(train[numeric_cols].median())
test[numeric_cols.drop('SalePrice')] = test[numeric_cols.drop('SalePrice')].fillna(test[numeric_cols.drop('SalePrice')].median())

categorical_cols = train.select_dtypes(include=['object']).columns
train[categorical_cols] = train[categorical_cols].fillna('None')
test[categorical_cols] = test[categorical_cols].fillna('None')

In [ ]:
train = pd.get_dummies(train, columns=categorical_cols, drop_first=True)
test = pd.get_dummies(test, columns=categorical_cols, drop_first=True)

train, test = train.align(test, join='left', axis=1, fill_value=0)

In [ ]:
X = train.drop(['Id', 'SalePrice'], axis=1)
y = np.log1p(train['SalePrice'])
X_test = test.drop(['Id'], axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
xgb = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, random_state=42)
xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("RMSE на валидационной выборке:", rmse)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_val, y_pred, alpha=0.5)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--')
plt.xlabel('Фактический LogSalePrice')
plt.ylabel('Предсказанный LogSalePrice')
plt.title('Предсказанные vs Фактические значения LogSalePrice (XGBoost)')
plt.show()

In [ ]:
rmse_cv = np.sqrt(-cross_val_score(xgb, X, y, cv=5, scoring='neg_mean_squared_error').mean())
print("Средний RMSE по 5-кратной кросс-валидации:", rmse_cv)

test_preds = np.expm1(xgb.predict(X_test))
submission = pd.DataFrame({'Id': test['Id'], 'SalePrice': test_preds})
submission.to_csv('submission.csv', index=False)
print("Файл submission.csv создан")